In [2]:
import pandas as pd
from tqdm import tqdm
import os,re

In [30]:
import queue
q1 = queue.LifoQueue([1,2,3])

In [3]:
df = pd.read_csv('./../data/conll/validation.conll', sep='\t')
df

,Id,Doc_Id,Sent_Id,Word,Tag
0,0.0,1.091375e+18,1.0,¿,O
1,1.0,1.091375e+18,1.0,Qué,O
2,2.0,1.091375e+18,1.0,probabilidad,O
3,3.0,1.091375e+18,1.0,hay,O
4,4.0,1.091375e+18,1.0,de,O
...,...,...,...,...,...
86994,88214.0,1.500171e+18,2500.0,me,O
86995,88215.0,1.500171e+18,2500.0,logro,O
86996,88216.0,1.500171e+18,2500.0,calmar,O
86997,88217.0,1.500171e+18,2500.0,dios,O


In [4]:
mentions = pd.read_csv('./../data/socialdisner_v3/mentions.tsv', sep='\t')
mentions

,tweets_id,begin,end,type,extraction
0,1357198223706894339,12,19,ENFERMEDAD,alergia
1,1357198223706894339,21,26,ENFERMEDAD,covid
2,1357198223706894339,29,34,ENFERMEDAD,gripe
3,1494382407398662147,17,25,ENFERMEDAD,DIABETES
4,1494382407398662147,114,122,ENFERMEDAD,diabetes
...,...,...,...,...,...
19420,1476675069069910025,178,185,ENFERMEDAD,Anginas
19421,1476675069069910025,238,246,ENFERMEDAD,ansiedad
19422,1405816276803596290,55,74,ENFERMEDAD,esclerosis múltiple
19423,1243769396096614400,54,62,ENFERMEDAD,ansiedad


In [6]:
# version 2 conll generation

path = './../data/socialdisner_v3/train-valid-txt-files/validation/'

fp = open("./../data/conll/testfromval1500_v2.conll", "w")
print("Id\tDoc_Id\tSent_Id\tWord\tTag\tbegin\tend", file=fp)
s = 1
for j,l in tqdm(enumerate(os.listdir(path))):
    #print(l[:-4])
    if j < 1000:
        continue
    doc_id = l[:-4]
    spans = mentions[mentions.tweets_id == int(l[:-4])]
    if len(spans) == 0:
        continue
    gold_start = list(spans.begin)
    gold_end = list(spans.end)
    #print('mentions:', spans)
    current_df = df[df.Doc_Id == int(l[:-4])]
    # changing word list
    
    f = open(f'{path+l}', 'r')
    info = f.read()
    #print('sent: ', info.strip())
    info = info.replace("\n", ' ')
    word_list = info.strip().split(' ')
    #print(len(word_list), len(set(word_list)))
    " create freq dict "
    #print(info)
    #print(word_list)
    c = 0
    tags = []
    annotations = []
    for w in word_list:
        if w == '':
            c +=1
            continue#w = '*'
        if c!= 0:
            c +=1
        start = c if (w[0] != '#') else c+1
        end = c+len(w)
        
        # tagging schema
        if start+1 in gold_start and len(w)>1:
            tag = 'B-MISC'
            
        elif start in gold_start:
            tag = 'B-MISC'
            
        elif (start not in gold_start) and (end in gold_end):
            tag = 'I-MISC'
            if len(gold_start) >1:
                if start > min(gold_start) and start < max(gold_start):
                    tag = 'B-MISC'
                elif start < min(gold_start):
                    tag = 'B-MISC'
            else:
                if start < max(gold_start):
                    tag = 'B-MISC'
                elif start == max(gold_start)+1:
                    tag = 'B-MISC'
            
        elif (start > max(gold_start)) and (end < max(gold_end)):
            tag = 'I-MISC'
            if start == max(gold_start)+1:
                tag = 'B-MISC'
        
        else:
            tag = 'O'
            
            if len(gold_start) ==1:
                if start < gold_start[0] and end > gold_end[0]:
                    tag = 'B-MISC'
                
            
        tags.append(tag)
        
            
        print(f'{s}\t{doc_id}\t{j+1}\t{w}\t{tag}\t{start}\t{end}', file=fp)
        annotations.append(f'{s}  {doc_id}  {j+1}  {w}  {tag}  {start}  {end}')
        s +=1
        c += len(w)
    
    print(file=fp)
    if 'B-MISC' not in tags:
        print(doc_id, tags, spans)
        print("check =====>")
        print(*annotations, sep='\n')
        print("*"*50)
    #if j == 10:
     #   break
fp.close()

2500it [00:01, 1331.08it/s]
